# PyTorch Pressure Drop Regression 

## Neural Network Architecture

In [1]:
# Adjustments: Implementing hyperparameter optimization
import torch 
import torch.nn as nn 

class DelpNN(nn.Module):
    """
    A simple neural network for regression with five fully connected layers and dropout regularization.
    """

    def __init__(self):
        """
        Initializes the neural network components.
        """
        super(DelpNN, self).__init__()  # Initialize the superclass
        # Define the layers of the neural network
        self.fc1 = nn.Linear(6, 256)  # First fully connected layer with 6 inputs and 256 outputs
        self.fc2 = nn.Linear(256, 128)  # Second fully connected layer with 256 inputs and 128 outputs
        self.fc3 = nn.Linear(128, 64)    # Third fully connected layer with 128 inputs and 64 outputs
        self.fc4 = nn.Linear(64, 16)    # Fourth fully connected layer with 64 inputs and 16 outputs
        self.fc5 = nn.Linear(16, 1)    # Fifth fully connected layer with 16 inputs and 1 output (regression)
        self.relu = nn.ReLU()  # Activation function used between layers
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with a dropout probability of 0.5

    def forward(self, x):
        """
        Defines the forward pass of the neural network.
        """
        x = self.relu(self.fc1(x))  # Pass input through the first layer and apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.relu(self.fc2(x))  # Pass through the second layer and apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.relu(self.fc3(x))  # Pass through the third layer and apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.relu(self.fc4(x))  # Pass through the fourth layer and apply ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = self.fc5(x)  # Output layer produces final output, no activation (linear output)
        return x

## Load and Prepare Data 

In [2]:
import pandas as pd

def load_and_prepare_data(filepath):
    """
    Load data from a file for training a machine learning model and convert it into tensors without scaling.
    
    Args:
        filepath (str): Path to the data file. This file should be a whitespace-delimited text file and
                        can contain comments starting with '%'.

    Returns:
        tuple: A tuple containing two elements:
            - X_tensor (Tensor): Tensor containing the raw features.
            - Y_tensor (Tensor): Tensor containing the raw target variable.
            Returns (None, None) if an error occurs during file loading or data processing.
    """
    try:
        # Read the file, ignoring lines with '%', which are comments.
        data = pd.read_csv(filepath, delim_whitespace=True, comment='%', header=None)
        
        # Set custom column names for ease of access
        data.columns = ['HCC', 'WCC', 'LCC', 'Tamb', 'Uin', 'Q',
                        'Temperature1', 'Temperature2', 'Temperature3', 'Temperature4',
                        'Temperature5', 'Temperature6', 'Pressure1', 'Pressure2', 'Delp',
                        'Velocity1', 'Velocity2', 'Tsta', 'StackTemp2']
    except Exception as e:
        print(f"Error reading the data file: {e}")
        return None, None

    # Select specific columns for features and target
    feature_labels = ['HCC', 'WCC', 'LCC', 'Tamb', 'Uin', 'Q']
    X = data[feature_labels]
    Y = data['Delp']
    
    return X, Y

## Split Data

In [3]:
from sklearn.model_selection import train_test_split

def split_data(X, Y, test_size=0.2, random_state=42):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )
    return X_train, X_test, Y_train, Y_test

## Preprocess Data

In [4]:
from sklearn.preprocessing import StandardScaler
import joblib
from torch.utils.data import DataLoader, TensorDataset

def preprocess_and_prepare_loaders(X_train, X_test, Y_train, Y_test, batch_size=64):
    scaler_X = StandardScaler()

    # Scale training data
    X_train_scaled = scaler_X.fit_transform(X_train)
    
    # Scale testing data using the same scaler
    X_test_scaled = scaler_X.transform(X_test)

    # Save the scaler for potential inverse transformations later
    joblib.dump(scaler_X, '6_scaler_X.pkl')

    # Convert scaled data to tensors. Use .values to correctly extract numpy arrays from pandas structures.
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

    # Ensure that Y_train and Y_test, if they are pandas Series, are converted to tensors properly.
    Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32).unsqueeze(1)
    Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32).unsqueeze(1)   # Use .values to extract the numpy array

    # Create TensorDataset and DataLoader
    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Print samples of scaled data for verification
    print("Scaled Training Features Sample:")
    print(X_train_tensor[:5])
    print("Training Targets Sample:")
    print(Y_train_tensor[:5])
    print("Scaled Testing Features Sample:")
    print(X_test_tensor[:5])
    print("Testing Targets Sample:")
    print(Y_test_tensor[:5])

    return train_loader, test_loader

## Train Model

In [5]:
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch
import os

def train_model(config, checkpoint_dir=None, data_dir=None):
    model = DelpNN()
    criterion = nn.L1Loss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["lr"],
        betas=(config["beta1"], config["beta2"]),
        eps=1e-08,
        weight_decay=config["weight_decay"],
    )
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    train_loader, test_loader = data_dir

    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(config["num_epochs"]):
        running_loss = 0.0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_loss = sum(criterion(model(inputs), labels).item() for inputs, labels in test_loader) / len(test_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
            with tune.checkpoint_dir(epoch) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint.pt")
                torch.save((model.state_dict(), optimizer.state_dict()), path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= config["patience"]:
                print(f'Early stopping after {epoch+1} epochs')
                break

        tune.report(loss=val_loss)
    
    print('Training complete.')

## Evaluate Model

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, test_loader, criterion, filename='true_vs_predicted.png'):
    """
    Evaluate the performance of a trained neural network model.

    Args:
        model (torch.nn.Module): The trained model.
        test_loader (DataLoader): DataLoader for the test data.
        criterion (torch.nn.MSELoss): Loss function to use for evaluating the model.
    """
    model.eval()  # Set the model to evaluation mode.
    total_loss = 0
    total_rmse = 0
    predictions, actuals = [], []

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)  # Compute model output

            # Calculate loss for the batch directly with tensors
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)

            # Convert tensors to numpy for calculating RMSE and plotting
            outputs_np = outputs.detach().cpu().numpy()
            labels_np = labels.detach().cpu().numpy()

            # Calculate RMSE for each batch
            rmse = mean_squared_error(labels_np, outputs_np, squared=False)
            total_rmse += rmse * labels.size(0)

            # Store predictions and actuals for plotting
            predictions.extend(outputs_np.flatten())
            actuals.extend(labels_np.flatten())

    # Compute the average loss and RMSE
    mean_loss = total_loss / len(test_loader.dataset)
    mean_rmse = total_rmse / len(test_loader.dataset)
    relative_error = (mean_rmse / np.mean(actuals)) * 100  # Calculate relative error as a percentage
    r2 = r2_score(actuals, predictions)  # Calculate R^2 score

    # Print calculated metrics
    print(f'Test MSE: {mean_loss:.4f}')
    print(f'Test RMSE: {mean_rmse:.4f} Pa')
    print(f'Relative Error: {relative_error:.2f}%')
    print(f'R^2 Score: {r2:.4f}')  # Print R^2 score

    # Plot true vs predicted values
    rcParams['font.family'] = 'serif'
    rcParams['font.serif'] = ['Times New Roman']
    rcParams['font.size'] = 12
    plt.figure(figsize=(10, 5))
    plt.scatter(actuals, predictions, alpha=0.5)
    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'k-', label='Ideal Line')
    plt.xlabel('True Pressure Drop')
    plt.ylabel('Predicted Pressure Drop')
    plt.grid(True)

    # Save the figure as a PNG file
    plt.savefig(filename, format='png', bbox_inches='tight')

    plt.show()
    
    return mean_rmse

## Predict New Data

In [7]:
def predict_single_data_point(data_point, model_path, scaler_path):
    # Load the trained model
    model = DelpNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode

    # Load the scaler
    scaler_X = joblib.load(scaler_path)

    # Scale the single data point
    data_point_scaled = scaler_X.transform([data_point])  # Note: [data_point] makes it 2D

    # Convert to tensor
    data_point_tensor = torch.tensor(data_point_scaled, dtype=torch.float32)

    # Make predictions
    with torch.no_grad():  # No need to track gradients for predictions
        output = model(data_point_tensor)

    prediction = output.numpy()  # Convert to numpy array if necessary

    return prediction

## Main 

In [8]:
def main():
    # Main function to run the training and evaluation process.
    filepath = '/Users/hanheelee/Desktop/Projects/00_ML_Fuel_Cell/Set 1.txt'  # Path to data file
    X, Y = load_and_prepare_data(filepath)
    if X is None or Y is None:
        print("Failed to load data.")
        return
    
    # Print samples for verification
    print("Sample X (features):")
    print(X[:10])
    print("Sample Y (targets):")
    print(Y[:10])
    
    # Split data
    X_train, X_test, Y_train, Y_test = split_data(X,Y)
    
    # Print samples for verification
    print("Sample of training features:")
    print(X_train.head())
    print("Sample of training targets:")
    print(Y_train.head())
    print("Sample of testing features:")
    print(X_test.head())
    print("Sample of testing targets:")
    print(Y_test.head())

    # Prepare data loaders
    train_loader, test_loader = preprocess_and_prepare_loaders(X_train, X_test, Y_train, Y_test)
    
    data_dir = (train_loader, test_loader)

    search_space = {
        "lr": tune.loguniform(1e-5, 1e-3),
        "beta1": tune.uniform(0.5, 0.9),
        "beta2": tune.uniform(0.9, 0.999),
        "weight_decay": tune.loguniform(1e-6, 1e-2),
        "num_epochs": tune.choice([50, 100, 200]),
        "patience": tune.choice([5, 10, 20])
    }

    
    bayesopt = BayesOptSearch()
    scheduler = ASHAScheduler(metric="loss", mode="min")

    analysis = tune.run(
        tune.with_parameters(train_model, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": 0},
        config=search_space,
        num_samples=10,
        scheduler=scheduler,
        search_alg=bayesopt
    )

    print("Best hyperparameters found were: ", analysis.best_config)
    
    # Load the best model checkpoint
    best_trial = analysis.get_best_trial("loss", "min", "last")
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint.pt"))

    # Create a new model instance and load the state
    best_model = DelpNN()
    best_model.load_state_dict(model_state)

    # Evaluate the best model
    mean_rmse = evaluate_model(best_model, test_loader, nn.L1Loss())

    # Save the best model's state dictionary
    filename = f'Delp_{mean_rmse:.4f}.pth'
    torch.save(best_model.state_dict(), filename)
    
    print("Model and scalers are saved successfully.")

if __name__ == "__main__":
    # Set fixed random number seed
    torch.manual_seed(42)
    ray.init()
    main()  # Execute the main function if the script is run directly.

2024-07-22 17:26:32,812	INFO worker.py:1788 -- Started a local Ray instance.
/var/folders/73/wzyp1fs57lg7p28crz696s_r0000gp/T/ipykernel_71372/85108997.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(filepath, delim_whitespace=True, comment='%', header=None)


Sample X (features):
     HCC     WCC   LCC    Tamb  Uin       Q
0  0.001  0.0005  0.03  253.15  1.0  1272.0
1  0.001  0.0005  0.03  273.15  1.0  1272.0
2  0.001  0.0005  0.03  293.15  1.0  1272.0
3  0.001  0.0005  0.03  313.15  1.0  1272.0
4  0.001  0.0005  0.03  253.15  1.0  3132.0
5  0.001  0.0005  0.03  273.15  1.0  3132.0
6  0.001  0.0005  0.03  293.15  1.0  3132.0
7  0.001  0.0005  0.03  313.15  1.0  3132.0
8  0.001  0.0005  0.03  253.15  1.0  5040.0
9  0.001  0.0005  0.03  273.15  1.0  5040.0
Sample Y (targets):
0     86.117
1     89.914
2     93.532
3     96.944
4     92.271
5     96.889
6    101.240
7    105.300
8     99.193
9    104.310
Name: Delp, dtype: float64
Sample of training features:
          HCC      WCC   LCC    Tamb    Uin       Q
3281  0.00200  0.00100  0.09  273.15   7.75  3132.0
2383  0.00175  0.00100  0.09  313.15   7.75  3132.0
2009  0.00175  0.00050  0.06  273.15   5.50  3132.0
2114  0.00175  0.00075  0.03  293.15   3.25  1272.0
1128  0.00125  0.00125  0.06 

2024-07-22 17:26:34,922	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-07-22 17:26:34,940	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2024-07-22 17:26:34,941	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.


ValueError: BayesOpt does not support parameters of type `Categorical`